In [1]:
import pandas as pd
import numpy as np

In [2]:
events = pd.read_excel('/content/drive/My Drive/Датасеты/Мероприятия/Мероприятия.xlsx')

In [3]:
given_books = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Выдача_1.xlsx') # предпочтения пользователя

In [4]:
given_books_too = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Выдача_2.xlsx')

In [5]:
catalog = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Каталог.xlsx') # жанры книг

In [6]:
readers = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Читатели.xlsx') # можно взять дату рождения

In [7]:
samples = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Экземпляры.xlsx') # бесполезно

In [8]:
samples_too = pd.read_excel('/content/drive/My Drive/Датасеты/Книги/Экземпляры_2.xlsx')

.

In [9]:
from datetime import datetime
today = datetime.today()

In [10]:
def ages(birthdate):
    if type(birthdate) == type(today):
        return (today-birthdate).days // 365.25
    else:
        return 0

In [11]:
readers['Возраст'] = readers['Дата рождения'].apply(ages)
readers = readers.drop(columns=['Дата рождения'])

In [41]:
given_books = given_books.append(given_books_too)
given_books = given_books.drop(columns=['ИД выдачи', 'Штрих-код', 'Дата выдачи', 'Дата сдачи (предполагаемая)', 'Состояние', 'Unnamed: 7', 'Состояния', 'Unnamed: 9'])
given_books = given_books.drop_duplicates()

In [50]:
readers_id = pd.unique(readers['ID читателя'])
reader_id = pd.unique(given_books['ИД читателя'])
ids = set(readers_id) & set(reader_id)

In [100]:
condition = readers['ID читателя'].apply(lambda x: x in ids)
readers = readers[condition]

In [104]:
condition = given_books['ИД читателя'].apply(lambda x: x in ids)
given_books = given_books[condition]

.

In [78]:
catalog = catalog.drop(columns=['p260a', 'p260b', 'p260c', 'p490a', 'p084a', 'p521a'])
catalog = catalog.fillna(0)

In [73]:
samples = samples_too.drop(columns=['ИД Каталожной записи', 'Штрих-код', 'Раздел знаний', 'Идентификатор сиглы'])

In [85]:
doc_id = pd.unique(catalog['doc_id'])
doc_ids = pd.unique(samples['Идентификатор экземпляра'])
common_ids = set(doc_id) & set(doc_ids)

In [90]:
condition = catalog['doc_id'].apply(lambda x: x in common_ids)
catalog = catalog[condition]

In [96]:
condition = samples['Идентификатор экземпляра'].apply(lambda x: x in common_ids)
samples = samples[condition]

.

In [108]:
invent_numbers = pd.unique(given_books['Инвентарный номер'])
in_numbers = pd.unique(samples['Инвентарный номер'])
common_invent = set(invent_numbers) & set(in_numbers)

In [118]:
cond = given_books['Инвентарный номер'].apply(lambda x: x in common_invent)
given_books = given_books[cond]

In [121]:
cond = samples['Инвентарный номер'].apply(lambda x: x in common_invent)
samples = samples[cond]

.

In [132]:
res = given_books.set_index('Инвентарный номер').join(samples.set_index('Инвентарный номер'))

In [146]:
res2 = res.set_index('Идентификатор экземпляра').join(catalog.set_index('doc_id'))

In [162]:
res3 = readers.merge(res2, left_on='ID читателя', right_on='ИД читателя').drop(columns=['ИД читателя'])

.

In [166]:
res3

,ID читателя,Возраст,p100a,p245a,p650a
0,28869,8.0,Греченко Татьяна Николаевна,Психофизиология: Учебно пособие,Психология личности
1,29217,5.0,0,Как на Дерибасовской угол Ришельевской...: Оче...,Художественная литература
2,28359,5.0,0,Cosmopolitan,0
3,28359,5.0,Савинкина Елена Владимировна,История химии. Элективный курс,Химические науки
4,39224,11.0,Остен Джейн,Гордость и предубеждение и зомби,Художественная литература
...,...,...,...,...,...
6239,12112,86.0,Бубеннов Михаил Семенович,Зарницы красного лета,Художественная литература
6240,19655,36.0,Лацис Вилис,Собрание сочинений,Художественная литература
6241,12338,57.0,0,Руководство чтением детей и юношества в библио...,Библиотечное дело. Библиотековедение
6242,19989,72.0,Тарыничев Юрий Федорович,На судском заломе,Рыбное хозяйство


In [167]:
autors = pd.unique(res3['p100a'])
authors_dict = {}
for i, a in enumerate(autors):
    authors_dict[a] = i
res3['author'] = res3['p100a'].apply(lambda x: authors_dict[x])

In [179]:
names = pd.unique(res3['p245a'])
names_dict = {}
for i, a in enumerate(names):
    names_dict[a] = i
res3['names'] = res3['p245a'].apply(lambda x: names_dict[x])

In [198]:
janr = pd.unique(res3['p650a'])
janr_dict = {}
for i, a in enumerate(janr):
    janr_dict[a] = i
res3['janr'] = res3['p650a'].apply(lambda x: janr_dict[x])

In [203]:
data = res3.drop(columns=['p100a', 'p245a', 'p650a'])

In [204]:
data

,ID читателя,Возраст,author,names,janr
0,28869,8.0,1,1,1
1,29217,5.0,0,2,2
2,28359,5.0,0,3,0
3,28359,5.0,2,4,4
4,39224,11.0,3,5,2
...,...,...,...,...,...
6239,12112,86.0,203,294,2
6240,19655,36.0,278,38,2
6241,12338,57.0,0,2947,238
6242,19989,72.0,1837,2916,25


In [223]:
from sklearn.neighbors import NearestNeighbors

In [224]:
clsf = NearestNeighbors()

In [225]:
clsf.fit(data)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)